# Experimental python-irodsclient

In [ ]:
pip install --upgrade python-irodsclient

Check if the module is available:

In [ ]:
import irods
irods.__version__

## Connect to SURF iRODS servers (YODA, NPEC, UNLOCK, ...)
To connect to an iROD instance you need to know the URL of the iRODS server, your user credentials and in som cases also som SSL settings. Usually iRODS connections are coommunicated by a json file like this:

```
{
    "irods_host": "wur-yoda.irods.hosting.com",
    "irods_port": 1247,
    "irods_user_name": "user@institute.com",
    "irods_zone_name": "wur",
    "irods_home": "/wur/home",
    "irods_authentication_scheme": "pam",
    "irods_encryption_num_hash_rounds": 16,
    "irods_client_server_policy": "CS_NEG_REQUIRE",
    "irods_encryption_algorithm": "AES-256-CBC",
    "irods_encryption_salt_size": 8,
    "irods_ssl_verify_server": "none",
    "irods_encryption_key_size": 32
}
```

In [ ]:
from irods.session import iRODSSession
import json

with open('/home/<USER>/.irods/irods_environment.json') as f:
    ienv = json.load(f)
with open('/home/<USER>/.irods/passwd') as f:
    passwd = f.readline()
session = iRODSSession(**ienv, password=passwd)
print(session.zone)

In [ ]:
coll = session.collections.get('/'+session.zone+'/home/user@institute.com')
print(coll.path)
print(coll.data_objects)
print(coll.subcollections)
print("Your current collection:", coll.path)
iHome = coll.path

# Metadata hooks for NPEC and UNLOCK
## 1. The resource metadata tag "free_space"

In [ ]:
from irods.models import Collection, DataObject, Resource, CollectionMeta, DataObjectMeta, ResourceMeta
from pprint import pprint

query = session.query(Resource.name, Resource.free_space)
print(query.all())


Both, bundleResc and demoResc have been disabled. One cannot put data into them:

In [ ]:
import irods.keywords as kw
options = {kw.RESC_NAME_KW: 'demoResc'}
%ls
iPath = iHome+'/testUpload.txt'
session.data_objects.put('hello.py', iPath, **options)

## 2. The data tag "RESOURCE"
The metadata key "RESOURCE" is reserved for data objects only and can only have the values 'disk', 'archive' and 'dual'. It moves the data objects' bitstreams to the respective data resource in iRODS.

In [ ]:
coll.metadata.add("RESOURCE", "disk")

In [ ]:
obj = session.data_objects.get(coll.path+"/demo.txt")
obj.metadata.items()

In [ ]:
obj.metadata.add("RESOURCE", "dual")
print(obj.metadata.items())
vars(obj)

In [ ]:
for item in obj.metadata.items():
    print(item)

You see that two metadata items are created with the key "RESOURCE". So metadata items are not updated but aggregated. In this case, remove the old metadata entry first and then create a new one with the new value:

In [ ]:
obj.metadata.remove("RESOURCE", "disk")
obj.metadata.remove("RESOURCE", "dual")
for item in obj.metadata.items():
    print(item)

In [ ]:
# Nonsense value
obj.metadata.add("RESOURCE", "hello")

In [ ]:
for item in obj.metadata.items():
    print(item)

In [ ]:
obj.metadata.add("RESOURCE", "archive")
for item in obj.metadata.items():
    print(item)